Introducción

# Predicción de Ventas en un E-commerce

**Objetivo:** Predecir cuántas unidades se venderán por tienda y producto en el próximo mes.

Este proyecto resuelve un problema real usando datos de la competición [Kaggle: Predict Future Sales](https://www.kaggle.com/competitions/competitive-data-science-predict-future-sales).

Se ha seguido un flujo completo de Machine Learning desde el análisis exploratorio hasta la generación de predicciones.


Carga de librerías y datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
import joblib
from sklearn.metrics import mean_squared_error
from math import sqrt


Exploración y limpieza - resumen

In [2]:
#Cargar datos limpios
df_sales = pd.read_csv('../data/sales_clean.csv')

#Mostrar estadísticas básicas
df_sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.928485e+06,2.928485e+06,2.928485e+06,2.928485e+06,2.928485e+06
mean,1.456976e+01,3.300294e+01,1.020028e+04,8.894673e+02,1.247598e+00
std,9.422951e+00,1.622543e+01,6.324392e+03,1.727500e+03,2.292948e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-02,1.000000e+00
25%,7.000000e+00,2.200000e+01,4.477000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.355000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.569100e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,1.000000e+03


- Se eliminaron precios negativos y valores extremos.
- Se eliminaron duplicados.

Agregación mensual

In [3]:
#Agrupar por mes, tienda y producto
df_monthly = df_sales.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()
df_monthly.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)
df_monthly['item_cnt_month'] = df_monthly['item_cnt_month'].clip(0, 20)

Creación de matriz base

In [4]:
#Matriz con todas las combinaciones mes-tienda-producto
df_matrix = pd.read_csv('../data/matriz_features_lags.csv')

Feature engineering

In [5]:
#Mostrar features clave creadas
df_matrix[['item_cnt_month', 'item_cnt_month_lag_1', 'item_cnt_month_lag_3']].describe()

,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_3
count,1.088455e+07,1.088455e+07,1.088455e+07
mean,2.996268e-01,2.833825e-01,2.594169e-01
std,1.225039e+00,1.201368e+00,1.161263e+00
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00
max,2.000000e+01,2.000000e+01,2.000000e+01


División en train / validación

In [6]:
df_train = df_matrix[df_matrix['date_block_num'] < 33]
df_val = df_matrix[df_matrix['date_block_num'] == 33]

features = [
    'item_id', 'shop_id', 'item_category_id',
    'month', 'year',
    'item_avg_monthly_sales', 'shop_avg_monthly_sales',
    'item_cnt_month_lag_1', 'item_cnt_month_lag_3'
]
target = 'item_cnt_month'

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]

Entrenamiento del modelo

In [7]:
model = XGBRegressor(
    n_estimators=100, max_depth=8,
    learning_rate=0.05, subsample=0.5,
    random_state=42, n_jobs=-1
)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

Evaluación

In [8]:
y_pred = model.predict(X_val)
rmse = sqrt(mean_squared_error(y_val, y_pred))
print(f'RMSE en validación: {rmse:.4f}')

RMSE en validación: 0.8958


Predicción final y generación de archivo

In [9]:
submission = pd.read_csv('../data/submission.csv')
submission.head()

,ID,item_cnt_month
0,0,0.924389
1,1,3.073612
2,2,0.918653
3,3,0.430294
4,4,5.292832


In [10]:
joblib.dump(model, '../models/xgb_model_final.joblib')

['../models/xgb_model_final.joblib']

### Conclusiones

- Se ha construido un modelo robusto con XGBoost.
- RMSE en validación: **0.8958**
- Se han aplicado técnicas de agregación temporal, lags y optimización.

